In [ ]:
from langchain_openai import ChatOpenAI

LLM_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507-FP8"
# LLM_ID = "Qwen/Qwen3-8B"
OPENAI_URL = "http://127.0.0.1:9805/v1"
TIMEOUT_SEC = 16

model = ChatOpenAI(
    api_key="EMPTY",
    base_url=OPENAI_URL,
    model=LLM_ID
)

In [ ]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# 할루시네이션 체크를 위한 데이터 모델 정의
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# 함수 호출을 통한 LLM 초기화
llm = ChatOpenAI(
    api_key="EMPTY",
    base_url=OPENAI_URL,
    model=LLM_ID
)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# 프롬프트 설정
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

# 프롬프트 템플릿 생성
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

# 환각 평가기 생성
hallucination_grader = hallucination_prompt | structured_llm_grader
